In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
uploaded.keys()

dict_keys(['subjective_test.txt'])

In [0]:
f = open('testxyz/textxyz/subjective_test.txt')

In [0]:
lines = f.readlines()
lines_and_classes = [(item, 0) for item in lines]


vile and tacky are the two best adjectives to describe ghost ship .



In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM

# model parameters:
num_char = 256
embeddings_dim = 50
lstm_memory = 32
num_classes = 2

model = Sequential()

model.add(Embedding(num_char,
                    embeddings_dim))
model.add(LSTM(lstm_memory))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 50)          12800     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 23,490
Trainable params: 23,490
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Convert lines into arrays of one hot vectors, and put them in same-size batches
import numpy as np
from collections import defaultdict
from keras.utils import to_categorical


def get_batches_from_lines_and_classes(lines_and_classes, batch_size=10):
  x_train_dict = defaultdict(list)
  y_train_dict = defaultdict(list)
  characters =[([ord(char) for char in line], to_categorical(c, num_classes))
                          for line, c in lines_and_classes]
  
  for item, c in characters:
    x_train_dict[len(item)].append(item)
    y_train_dict[len(item)].append(c)

  batches = []
  for k in x_train_dict.keys():
    x_train = np.array(x_train_dict[k])
    y_train = np.array(y_train_dict[k])
    x_chunks = [x_train[x:x + batch_size] for x in range(0, len(x_train), batch_size)]
    y_chunks = [y_train[y:y + batch_size] for y in range(0, len(y_train), batch_size)]
    batches.append([(x, y) for x, y in zip(x_chunks, y_chunks)])
    
  return batches

In [0]:
%%time
import random
from tqdm import tqdm

epochs = 5

batches = get_batches_from_lines_and_classes(lines_and_classes)
batches = batches[:5]
for i in range(epochs):
  print('Epoch' + str(i))
  random.shuffle(batches)
  for batch in batches:
    x_train = batch[0][0]
    y_train = batch[0][1]
    model.train_on_batch(x_train, y_train)
  model.save('model' + str(i) + '.keras')

Epoch0
Epoch1
Epoch2
Epoch3
Epoch4
CPU times: user 12.6 s, sys: 837 ms, total: 13.5 s
Wall time: 11.8 s


In [0]:
from keras.models import load_model

loaded_model = load_model('model1.keras')

In [0]:
from keras.models import load_model

batches = get_batches_from_lines_and_classes(lines_and_classes)
batches = batches[:5]
for i in range(epochs):
  print('Testing Epoch' + str(i))
  load_model('model' + str(i) + '.keras')
  tot_accuracy = 0
  tot_items = 0
  for batch in batches:
    x_train = batch[0][0]
    y_train = batch[0][1]
    history = model.test_on_batch(x_train, y_train)
    num_items = x_train.shape[0]
    tot_accuracy += history[1] * num_items
    tot_items += num_items
    
  print(tot_accuracy/tot_items)
  

Testing Epoch0
1.0
Testing Epoch1


KeyboardInterrupt: ignored

In [0]:
!git clone https://github.com/fractalego/testxyz.git

Cloning into 'testxyz'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [0]:
# Print out the embeddings

from keras import backend as K

batches = get_batches_from_lines_and_classes(lines_and_classes, batch_size=1)
batches = batches[:5]

load_model('model' + str(0) + '.keras')
tot_accuracy = 0
tot_items = 0
embeddings = []
for batch in batches:
  x_train = batch[0][0]
  get_layer_output = K.function([model.layers[0].input],
                                [model.layers[-3].output])
  layer_output = get_layer_output([x_train])[0]
  embeddings.append(layer_output[0])

print(embeddings[0].shape)

(32,)
